# Phase II Model With Real Data
First attempt at a Phase II model, one which creases the master schedule and assigns students to courses simultaneously.

$S$ -- Set of all students

$C$ -- Set of all courses

$T$ -- Set of all periods {1,2,3,4,7,8} 

$I$ -- Set of all instructors

$R$ -- set of all rooms 

**Variables:**

$x_{i,j}$ for  $i \in S, j \in C$ -- Binary, 1 if student $i$ assigned to course $j$ 

$c_{j,t}$ for $j \in C, t \in T$ -- Binary, 1 if course $j$ to be offered in period $t$

$r_{j,s,t}$ for $j \in C$, $s \in R$ and $t \in T$ -- binary, describes if course $j$ to be held in room $s$ durring period $t$.

** Parameters:**

$P_{i,j}$ -- Preference for student $i$ on course $j$

$S_i$ -- Seniority constant, e.g., higher for seniors

$D_{i,j}$ -- Binary, 1 if course $i$ and $j$ are in the same department, i.e., if they meet the same requirement, e.g., highschool math

$Ta_{i,j}$ -- Binary, 1 if teacher $i$ is teaching course $j$

$Cap_j$ -- Capacity of course $j$

$Min_j$ -- Minimum number of students needed for course $j$

$Db_j$ -- Indicates if course $j$ is a double period (1 or 0)


** Constraints: **

$\sum_{j} x_{i,j} =6 \quad \forall i \in S$ -- Says students can be assigned to two courses (full course load).

**THIS IS NOT LINEAR!!!**  
$\sum_{j} x_{i,j}c_{j,t} = 1 \quad \forall i \in S, t \in T$ -- Students assigned to at most one course per period ).

$\sum_{i \in C} \sum_{j \neq i \in C} D_{i,j} x_{k,i} = 0 \quad \forall k \in S$ -- Says that students are not assigned to more than 1 class in each department (could always change to more nuanced number of courses per department)

$\sum_{t \in T} c_{j,t} = 1 \quad \forall j \in C$ -- Says that each course taught only once

$\sum_{i \in S} x_{i,j} \leq Cap_j \quad \forall j \in C$ -- Course capacity constraint

$\sum_{j \in C} c_{j,t} Ta_{k,j} \leq 1  \quad \forall k \in I, \forall t \in T$ -- Teacher constraint (a teacher can teach at most one course per period), where $Ta_{k,j}$ is a parameter, not a variable

$(2 - Db_j - c_{j,t}) + c_{j+1, t+1} \geq 1 \; \forall j, t$ -- Says that if a course is a double period course, and it is taugh in period $t$ then $j+1$ (its second half) must be taught in period $t+1$. If it is not a double period, or not taught in $t$ then the big $M$ makes the constraint trivial. (This assumes that if $j$ is a double period, then in the course document $j+1$ is its second half.

$(2 - Db_j - x_{i,j}) + x_{i,j+1} \geq 1 \; \forall i,j$ -- Says that if a course is a double period, and a student is enrolled in the first half, then they must also be enrolled in the second half. 

$(1-c_{j,t}) + \sum_{s \in R}r_{j,s,t} \geq 1$ for $j \in C$ and $t \in T$ -- Says that if course $j$ is offered in period $t$ then course $j$ must be assigned to *at least* one room durring that period.

$\sum_{s \in R}r_{j,s,t} \leq 1$ for $j \in C$ and $t \in T$ -- For a given course in a given period, it must be assigned at most one room (could be 0 if this course is not offered). 

The above two constraints should ensure that if a course is offered in period $t$ then it is assigned *exactly* one room. This step is required to essentially tie $c_{j,t}$ to $r_{j,s,t}$.
    
$\sum_{j \in C} r_{j,s,t} \leq 1 $ for $s \in R$ and $t \in T$ -- Each room can have at most one class in it at a time (we should make the gym two different rooms).

TODO: Formulate room constraint for departnemnts, i.e., at most 3 science courses per period.

** Objective: **

$ \text{max }\sum_{i \in S} \sum_{j \in C} x_{i,j} P_{i,j} $ -- Assuming preferences take a higher value if they are a student's preferred choice, this will give a higher weight to higher assignments (at this point, I am leaving out the seniority multiplier).

In [1]:
from pyscipopt import Model, quicksum
import numpy as np
import pandas as pd

In [2]:
# read in data
prefs = pd.read_csv("Resources/FlatChoicesBinary.csv")
courses = pd.read_csv("Resources/FlatCourseSize.csv")
prox = pd.read_csv("Resources/Proximity.csv")

# clean it up
prefs.rename(columns={"Unnamed: 0": "Student"}, inplace=True)
courses.rename(columns={"0":"Class"}, inplace=True)
courses.drop("Unnamed: 0", axis=1, inplace=True)

In [3]:
# Extract sets
S = prefs["Student"].tolist() # list of all students (once we get ID make dictionary)

C = {} # Course dictionary
for i in courses.index:
    C[i] = courses["Class"].iloc[i]
    
T = [1,2,3,4,7,8] # Periods

## STILL NEED INSTRUCTORS

In [4]:
# Extract Preferences
P = prefs.drop("Student", axis=1).as_matrix()

In [5]:
# Double periods
Db = courses["Double"].fillna(0).astype(int)

In [6]:
# Proximity Matrix
D = prox.drop("0", axis=1).as_matrix()

In [7]:
# Course Sizes (min and max)
MIN = courses["Min"]
MAX = courses["Max"]

# To check feasibility:
MIN = [0]*len(C)
MAX = [100]*len(C)


In [8]:
# Need instructor data

In [75]:
# Setup model
m = Model("PhaseTwo")

In [76]:
# Trackers--to verify what SCIP says
num_vars = 0
num_cons = 0

In [77]:
# Add Student Variables (X)
X = {}
for i in S:
    for j in range(len(C)):
        name = "Student " + str(i) + " in course " + str(j)
        X[i,j] = m.addVar(name, vtype='B')
        num_vars += 1

In [78]:
# Add Course Variable
Course = {} # Variable dictionary
for j in range(len(C)):
    for t in T:
        name = "Course " + str(j) + " in period " + str(t)
        Course[j,t] = m.addVar(name, vtype='B')
        num_vars += 1

In [79]:
# Add Student assignment constraint (must have two classes)
for i in S:
        m.addCons(quicksum(X[i,j] for j in C) == 6) # one per period
        num_cons += 1

In [26]:
# Add student period constraint
for i in S:
    for t in T:
        m.addCons(quicksum(X[i,j]*Course[j,t] for j in C) <= 1)
        num_cons += 1

In [80]:
# Add capacity and minimum constraint
for j in range(len(C)):
    m.addCons(quicksum(X[i,j] for i in S) <= MAX[j])
    #m.addCons(quicksum(X[i,j] for i in S) >= 0)
    num_cons += 2

In [81]:
# Add course proximity constraint (without quicksum)
Total = len(S) + len(C) + len(C) -1
l = 0
if not np.array_equal(D, np.zeros(np.array(D).shape)):
    for k in S:
        expr = 0 # reset expression (should not have impact)
        for i in C:
            small_set = list(set(C) - set([int(i)])) # C - {i} list of courses without course i
            for j in small_set:
                expr = expr + D[i][j]*X[k,i]
        
        m.addCons(expr <= 10) # could swap this to >= or = if they must take "at least" one course
        l += 1
        if l % 50 ==0:
            print(str(l), "of", str(len(S)))
print("Done")

50 of 284
100 of 284
150 of 284
200 of 284
250 of 284
Done


In [82]:
## REQUIRES TEACHER DATA
# Teacher Constraint
# for k in I:
#     for t in T:
#         m.addCons(quicksum(Course[j,t]*Ta[k][j] for j in C) == 1)
#         num_cons += 1

In [83]:
# Course Taught only once Constraint
for j in range(len(C)):
    m.addCons(quicksum(Course[j,t] for t in T) == 1)
    num_cons += 1

In [84]:
# Double Period--Consecutive Constraint
for j in range(len(C))[:-1]:
    for t in T[:-1]: # need the :-1 to ensure don't go over bounds below
        if t != 4 and t != 8:
            m.addCons(2 - Db[j] - Course[j,t] + Course[j+1, t+1] >= 1)

In [85]:
# Double Period Constraint--Students enrolled in both
for i in S:
    for j in range(len(C))[:-1]:
        m.addCons(2 - Db[j] - X[i,j] + X[i, j+1] >= 1)

In [86]:
## NEED ROOM DATA
# Room Constraint--Assigned to at least one room
# for j in C:
#     for t in T:
#         m.addCons(1 - Course[j,t] + quicksum(r[j,s,t] for s in R) >= 1)

In [87]:
## NEED ROOM DATA
# Room Constraint--Assigned to at most one room
# for j in C:
#     for t in T:
#         m.addCons(quicksum(r[j,s,t] for s in R) <= 1)

In [88]:
## NEED ROOM DATA
# Room Constraint--Each room gets at most one course
# for s in R:
#     for t in T:
#         m.addCons(quicksum(r[j,s,t] for j in C) <= 1)

In [89]:
# Set objective
m.setObjective(quicksum(X[i,j]*P[i][j] for i in S for j in C), "maximize")
#m.setObjective(X[1,1]*0, "maximize") # just find a feasible solution

In [90]:
print(str(num_vars), "Variables")
print(str(num_cons), "Constraints")

27550 Variables
569 Constraints


In [91]:
# Solve model
m.optimize() # NOTE: solver info printed to terminal

In [22]:
## TODO: Upadte output for large scale model
# Look at output
if m.getStatus() == "optimal":
    # Display which courses each student is assigned to
    for i in S:
        for j in C:
            s = str(X[i,j])
            if m.getVal(X[i,j]) == 1:
                print(s + ":\tyes")
            else:
                print(s + ":\tno")
                #pass
        print("\n")

    # Display which periods courses are assigned to
    for j in C:
        for t in T:
            s = str(Course[j,t])
            if m.getVal(Course[j,t]) == 1:
                print("Course " + str(j) + " to be taught in period", str(t))
                
    # Display the enrollment totals for each course
    print("\nCapacities:")
    for j in C:
        size = 0
        for i in S:
            if m.getVal(X[i,j]) == 1:
                size += 1
        #print("Course", j, "has", size, "seats filled of a possible", Cap[j])
        print("Course", str(j)+":", str(size) + "/" + str(MAX[j]))
        
    # Display the period in which the course is taught:
    print("\nPeriods")
    for j in C:
        for t in T:
            if m.getVal(Course[j,t])==1:
                print("Course", j, "taught in period", t)

else:
    print("The model is", m.getStatus())

Student 0 in course 0:	no
Student 0 in course 1:	no
Student 0 in course 2:	no
Student 0 in course 3:	no
Student 0 in course 4:	no
Student 0 in course 5:	no
Student 0 in course 6:	no
Student 0 in course 7:	no
Student 0 in course 8:	no
Student 0 in course 9:	no
Student 0 in course 10:	no
Student 0 in course 11:	no
Student 0 in course 12:	no
Student 0 in course 13:	no
Student 0 in course 14:	no
Student 0 in course 15:	no
Student 0 in course 16:	no
Student 0 in course 17:	no
Student 0 in course 18:	no
Student 0 in course 19:	no
Student 0 in course 20:	no
Student 0 in course 21:	no
Student 0 in course 22:	no
Student 0 in course 23:	no
Student 0 in course 24:	no
Student 0 in course 25:	no
Student 0 in course 26:	no
Student 0 in course 27:	no
Student 0 in course 28:	no
Student 0 in course 29:	no
Student 0 in course 30:	no
Student 0 in course 31:	no
Student 0 in course 32:	no
Student 0 in course 33:	no
Student 0 in course 34:	no
Student 0 in course 35:	no
Student 0 in course 36:	no
Student 0 i

Student 10 in course 47:	no
Student 10 in course 48:	no
Student 10 in course 49:	no
Student 10 in course 50:	no
Student 10 in course 51:	no
Student 10 in course 52:	no
Student 10 in course 53:	no
Student 10 in course 54:	no
Student 10 in course 55:	no
Student 10 in course 56:	no
Student 10 in course 57:	no
Student 10 in course 58:	no
Student 10 in course 59:	no
Student 10 in course 60:	no
Student 10 in course 61:	no
Student 10 in course 62:	no
Student 10 in course 63:	no
Student 10 in course 64:	no
Student 10 in course 65:	yes
Student 10 in course 66:	yes
Student 10 in course 67:	yes
Student 10 in course 68:	yes
Student 10 in course 69:	yes
Student 10 in course 70:	yes


Student 11 in course 0:	no
Student 11 in course 1:	no
Student 11 in course 2:	no
Student 11 in course 3:	no
Student 11 in course 4:	no
Student 11 in course 5:	no
Student 11 in course 6:	no
Student 11 in course 7:	no
Student 11 in course 8:	no
Student 11 in course 9:	no
Student 11 in course 10:	no
Student 11 in course 1

Student 31 in course 32:	no
Student 31 in course 33:	no
Student 31 in course 34:	no
Student 31 in course 35:	no
Student 31 in course 36:	no
Student 31 in course 37:	no
Student 31 in course 38:	no
Student 31 in course 39:	no
Student 31 in course 40:	no
Student 31 in course 41:	no
Student 31 in course 42:	no
Student 31 in course 43:	no
Student 31 in course 44:	no
Student 31 in course 45:	no
Student 31 in course 46:	no
Student 31 in course 47:	no
Student 31 in course 48:	no
Student 31 in course 49:	no
Student 31 in course 50:	no
Student 31 in course 51:	no
Student 31 in course 52:	no
Student 31 in course 53:	no
Student 31 in course 54:	no
Student 31 in course 55:	no
Student 31 in course 56:	no
Student 31 in course 57:	no
Student 31 in course 58:	no
Student 31 in course 59:	yes
Student 31 in course 60:	yes
Student 31 in course 61:	yes
Student 31 in course 62:	yes
Student 31 in course 63:	yes
Student 31 in course 64:	yes
Student 31 in course 65:	no
Student 31 in course 66:	no
Student 31 in 

Student 51 in course 38:	no
Student 51 in course 39:	no
Student 51 in course 40:	no
Student 51 in course 41:	no
Student 51 in course 42:	no
Student 51 in course 43:	no
Student 51 in course 44:	no
Student 51 in course 45:	no
Student 51 in course 46:	no
Student 51 in course 47:	no
Student 51 in course 48:	no
Student 51 in course 49:	no
Student 51 in course 50:	no
Student 51 in course 51:	no
Student 51 in course 52:	no
Student 51 in course 53:	yes
Student 51 in course 54:	yes
Student 51 in course 55:	yes
Student 51 in course 56:	yes
Student 51 in course 57:	yes
Student 51 in course 58:	yes
Student 51 in course 59:	no
Student 51 in course 60:	no
Student 51 in course 61:	no
Student 51 in course 62:	no
Student 51 in course 63:	no
Student 51 in course 64:	no
Student 51 in course 65:	no
Student 51 in course 66:	no
Student 51 in course 67:	no
Student 51 in course 68:	no
Student 51 in course 69:	no
Student 51 in course 70:	no


Student 52 in course 0:	no
Student 52 in course 1:	no
Student 52 in 

Student 72 in course 30:	no
Student 72 in course 31:	no
Student 72 in course 32:	no
Student 72 in course 33:	no
Student 72 in course 34:	no
Student 72 in course 35:	no
Student 72 in course 36:	no
Student 72 in course 37:	no
Student 72 in course 38:	no
Student 72 in course 39:	no
Student 72 in course 40:	no
Student 72 in course 41:	no
Student 72 in course 42:	no
Student 72 in course 43:	no
Student 72 in course 44:	no
Student 72 in course 45:	no
Student 72 in course 46:	no
Student 72 in course 47:	no
Student 72 in course 48:	yes
Student 72 in course 49:	yes
Student 72 in course 50:	yes
Student 72 in course 51:	yes
Student 72 in course 52:	yes
Student 72 in course 53:	no
Student 72 in course 54:	no
Student 72 in course 55:	no
Student 72 in course 56:	yes
Student 72 in course 57:	no
Student 72 in course 58:	no
Student 72 in course 59:	no
Student 72 in course 60:	no
Student 72 in course 61:	no
Student 72 in course 62:	no
Student 72 in course 63:	no
Student 72 in course 64:	no
Student 72 in 

...

Student 194 in course 20:	no
Student 194 in course 21:	no
Student 194 in course 22:	no
Student 194 in course 23:	yes
Student 194 in course 24:	yes
Student 194 in course 25:	yes
Student 194 in course 26:	yes
Student 194 in course 27:	yes
Student 194 in course 28:	no
Student 194 in course 29:	no
Student 194 in course 30:	no
Student 194 in course 31:	no
Student 194 in course 32:	no
Student 194 in course 33:	no
Student 194 in course 34:	no
Student 194 in course 35:	no
Student 194 in course 36:	no
Student 194 in course 37:	no
Student 194 in course 38:	no
Student 194 in course 39:	no
Student 194 in course 40:	no
Student 194 in course 41:	no
Student 194 in course 42:	no
Student 194 in course 43:	no
Student 194 in course 44:	no
Student 194 in course 45:	no
Student 194 in course 46:	no
Student 194 in course 47:	no
Student 194 in course 48:	no
Student 194 in course 49:	no
Student 194 in course 50:	no
Student 194 in course 51:	no
Student 194 in course 52:	no
Student 194 in course 53:	no
Student 1

Student 217 in course 61:	no
Student 217 in course 62:	no
Student 217 in course 63:	no
Student 217 in course 64:	no
Student 217 in course 65:	no
Student 217 in course 66:	no
Student 217 in course 67:	no
Student 217 in course 68:	no
Student 217 in course 69:	no
Student 217 in course 70:	no


Student 218 in course 0:	no
Student 218 in course 1:	no
Student 218 in course 2:	no
Student 218 in course 3:	no
Student 218 in course 4:	no
Student 218 in course 5:	no
Student 218 in course 6:	no
Student 218 in course 7:	no
Student 218 in course 8:	no
Student 218 in course 9:	no
Student 218 in course 10:	no
Student 218 in course 11:	no
Student 218 in course 12:	no
Student 218 in course 13:	no
Student 218 in course 14:	no
Student 218 in course 15:	no
Student 218 in course 16:	no
Student 218 in course 17:	no
Student 218 in course 18:	yes
Student 218 in course 19:	yes
Student 218 in course 20:	yes
Student 218 in course 21:	yes
Student 218 in course 22:	yes
Student 218 in course 23:	no
Student 218 in co

Student 236 in course 41:	no
Student 236 in course 42:	no
Student 236 in course 43:	no
Student 236 in course 44:	no
Student 236 in course 45:	no
Student 236 in course 46:	no
Student 236 in course 47:	no
Student 236 in course 48:	no
Student 236 in course 49:	no
Student 236 in course 50:	no
Student 236 in course 51:	no
Student 236 in course 52:	no
Student 236 in course 53:	no
Student 236 in course 54:	no
Student 236 in course 55:	no
Student 236 in course 56:	no
Student 236 in course 57:	no
Student 236 in course 58:	no
Student 236 in course 59:	no
Student 236 in course 60:	no
Student 236 in course 61:	no
Student 236 in course 62:	no
Student 236 in course 63:	no
Student 236 in course 64:	no
Student 236 in course 65:	no
Student 236 in course 66:	no
Student 236 in course 67:	no
Student 236 in course 68:	no
Student 236 in course 69:	no
Student 236 in course 70:	no


Student 237 in course 0:	yes
Student 237 in course 1:	yes
Student 237 in course 2:	yes
Student 237 in course 3:	yes
Student 237 

Student 260 in course 21:	no
Student 260 in course 22:	no
Student 260 in course 23:	no
Student 260 in course 24:	no
Student 260 in course 25:	no
Student 260 in course 26:	no
Student 260 in course 27:	no
Student 260 in course 28:	no
Student 260 in course 29:	no
Student 260 in course 30:	no
Student 260 in course 31:	no
Student 260 in course 32:	no
Student 260 in course 33:	no
Student 260 in course 34:	no
Student 260 in course 35:	no
Student 260 in course 36:	no
Student 260 in course 37:	no
Student 260 in course 38:	no
Student 260 in course 39:	no
Student 260 in course 40:	no
Student 260 in course 41:	no
Student 260 in course 42:	no
Student 260 in course 43:	no
Student 260 in course 44:	no
Student 260 in course 45:	no
Student 260 in course 46:	no
Student 260 in course 47:	no
Student 260 in course 48:	no
Student 260 in course 49:	no
Student 260 in course 50:	no
Student 260 in course 51:	no
Student 260 in course 52:	no
Student 260 in course 53:	no
Student 260 in course 54:	no
Student 260 in

Student 280 in course 37:	no
Student 280 in course 38:	no
Student 280 in course 39:	no
Student 280 in course 40:	no
Student 280 in course 41:	no
Student 280 in course 42:	no
Student 280 in course 43:	no
Student 280 in course 44:	no
Student 280 in course 45:	no
Student 280 in course 46:	no
Student 280 in course 47:	no
Student 280 in course 48:	no
Student 280 in course 49:	no
Student 280 in course 50:	no
Student 280 in course 51:	no
Student 280 in course 52:	no
Student 280 in course 53:	no
Student 280 in course 54:	no
Student 280 in course 55:	no
Student 280 in course 56:	yes
Student 280 in course 57:	no
Student 280 in course 58:	no
Student 280 in course 59:	no
Student 280 in course 60:	no
Student 280 in course 61:	no
Student 280 in course 62:	no
Student 280 in course 63:	no
Student 280 in course 64:	no
Student 280 in course 65:	no
Student 280 in course 66:	no
Student 280 in course 67:	no
Student 280 in course 68:	no
Student 280 in course 69:	no
Student 280 in course 70:	no


Student 281

In [92]:
# Print Information on Solve
m.printStatistics() # NOTE: this will only print to terminal (note notebook)

## Current issues:
- Missing data on:
    - Teachers
    - Course Proximity (departments)
    - Updates to accomodate:
        - Double period courses
        - Multiple instances of same course
        - "Other" in every peoriod
- Proximity constraint may be over complicated (the double sum involving $D$)
    

# TODO
- Room constraints for departments, i.e., no more than 3 science courses per period. 
- Code all Room and Double period constraints
- Enhance output of model to handle large-scale data

The following block of code is meant to help better understand SCIP. It instantiates a Model instance, then parses through each of its public methods and fields, looking for, and printing, their docstrings.

In [8]:
# Figure out wtf SCIP is doing

# initialize a Model instance
mod = Model("what?")

# get methods for the model
methods = dir(mod)
i = 0
for x in methods:
    if x[0]=="_":
        i = methods.index(x)
methods = methods[i+1:] # only want the public methods

# print out each method and its info
for m in methods:
    print(m + ":")
    print(getattr(mod, m).__doc__)
    print("\n")

addCons:
Add a linear or quadratic constraint.

        Keyword arguments:
        cons -- list of coefficients
        name -- the name of the constraint (use generic name if empty)
        initial -- should the LP relaxation of constraint be in the initial LP? (default True)
        separate -- should the constraint be separated during LP processing? (default True)
        enforce -- should the constraint be enforced during node processing? (default True)
        check -- should the constraint be checked for feasibility? (default True)
        propagate -- should the constraint be propagated during node processing? (default True)
        local -- is the constraint only valid locally? (default False)
        modifiable -- is the constraint modifiable (subject to column generation)? (default False)
        dynamic -- is the constraint subject to aging? (default False)
        removable -- hould the relaxation be removed from the LP due to aging or cleanup? (default False)
        stick